This notebook is a prototype of the rTMS Bayesian Optimization algorithm and will reflect the final step of pipeline after tetrahedral projection and FEM surface construction/registration

In [1]:
import os
import numpy as np
from fieldopt import geolib
from fieldopt.objective import FieldFunc

In [2]:
#MOE package loading (should wrap!)
from moe.optimal_learning.python.cpp_wrappers.domain import TensorProductDomain as cTensorProductDomain
from moe.optimal_learning.python.python_version.domain import TensorProductDomain
from moe.optimal_learning.python.geometry_utils import ClosedInterval
from moe.optimal_learning.python.cpp_wrappers.expected_improvement import ExpectedImprovement
from moe.optimal_learning.python.cpp_wrappers.expected_improvement import multistart_expected_improvement_optimization as meio
from moe.optimal_learning.python.data_containers import HistoricalData, SamplePoint
from moe.optimal_learning.python.cpp_wrappers.log_likelihood_mcmc import GaussianProcessLogLikelihoodMCMC
from moe.optimal_learning.python.default_priors import DefaultPrior
from moe.optimal_learning.python.python_version.optimization import GradientDescentOptimizer, GradientDescentParameters
from moe.optimal_learning.python.cpp_wrappers.optimization import GradientDescentOptimizer as cGDOpt
from moe.optimal_learning.python.cpp_wrappers.optimization import GradientDescentParameters as cGDParams

In [3]:
#Specify inputs
proj_dir =  '/projects/jjeyachandra/rtms_optimize'
mesh_file = os.path.join(proj_dir,'data','simnibs_output','sub-CMH090.msh')
coil_file = os.path.join(proj_dir,'resources','coils','Magstim_70mm_Fig8.nii.gz')
tet_file = os.path.join(proj_dir,'output','tetra_parcels')
C_file = os.path.join(proj_dir,'output','quadratic_vec')
iR_file = os.path.join(proj_dir,'output','inverse_rot')
b_file = os.path.join(proj_dir,'output','param_bounds')
testing_dir = '/tmp/'#os.path.join(proj_dir,'testing','tmp')

In [4]:
#Step 1: Load in files containing relevant information about search domain
C = np.fromfile(C_file)
iR = np.fromfile(iR_file).reshape(3,3)
b = np.fromfile(b_file).reshape(3,2)

#Load in element mask
p_map = np.load(os.path.join(proj_dir,'output','tetra_parcels.npy'))[:,2]

In [5]:
#Step 2: Specify the search domain
search_domain = TensorProductDomain([
    ClosedInterval(b[0,0],b[0,1]), #X coord on quadratic surface
    ClosedInterval(b[1,0],b[1,1]), #Y coord on quadratic surface
    ClosedInterval(0,180) #Rotational angle
])

c_search_domain = cTensorProductDomain([
    ClosedInterval(b[0,0],b[0,1]), 
    ClosedInterval(b[1,0],b[1,1]),
    ClosedInterval(0,180)
])

In [6]:
#Make objective function
f = FieldFunc(mesh_file=mesh_file, quad_surf_consts=C,
              surf_to_mesh_matrix=iR, tet_weights=p_map,
              field_dir=testing_dir, coil=coil_file)

In [7]:
#Generate historical data
hist_pts = 5
init_pts = search_domain.generate_uniform_random_points_in_domain(hist_pts)
observations = [-f.evaluate(x,y,t) for x,y,t in init_pts]

[ simnibs ]INFO: Could not find the dti tensor file in the standard location: /projects/jjeyachandra/rtms_optimize/data/simnibs_output/d2c_sub-CMH090/dti_results_T1space/DTI_conf_tensor.nii.gz
[ simnibs ]INFO: Head Mesh: /projects/jjeyachandra/rtms_optimize/data/simnibs_output/sub-CMH090.msh
[ simnibs ]INFO: Subject Path: /projects/jjeyachandra/rtms_optimize/data/simnibs_output/m2m_sub-CMH090
[ simnibs ]INFO: Simulation Folder: /tmp/tmpwecVuQ
[ simnibs ]INFO: Running simulations in the directory: /tmp/tmpwecVuQ
[ simnibs ]INFO: Running Poslist Number: 1
[ simnibs ]INFO: Began to run TMS simulations
[ simnibs ]INFO: Using coil: /projects/jjeyachandra/rtms_optimize/resources/coils/Magstim_70mm_Fig8.nii.gz
[ simnibs ]INFO: Using isotropic conductivities


0


[ simnibs ]INFO: Running TMS Position 0
[ simnibs ]INFO: Coil Position Matrix: [[ -0.72155797  -0.14971264   0.67597353 -52.19862584]
 [ -0.42846282  -0.67036766  -0.60582738  71.19690807]
 [ -0.54385081   0.7267691   -0.41956283  29.46282701]
 [  0.           0.           0.           1.        ]]
dIdt: 1000000.0

[ simnibs ]INFO: Executing: 
/mnt/tigrlab/projects/jjeyachandra/simnibs/bin/getdp /tmp/tmpwecVuQ/sub-CMH090_TMS_1-0001_Magstim_70mm_Fig8_nii.pro -solve QS -msh /tmp/tmpwecVuQ/tmp_LoYc3.msh -ksp_type cg -ksp_rtol 1e-10 -pc_type icc -pc_factor_levels 2 -v2 -bin
[ simnibs ]INFO: Info    : Running '/mnt/tigrlab/projects/jjeyachandra/simnibs/bin/getdp /tmp/tmpwecVuQ/sub-CMH090_TMS_1-0001_Magstim_70mm_Fig8_nii.pro -solve QS -msh /tmp/tmpwecVuQ/tmp_LoYc3.msh -ksp_type cg -ksp_rtol 1e-10 -pc_type icc -pc_factor_levels 2 -v2 -bin' [GetDP 2.11.3, 1 node, max. 1 thread]
[ simnibs ]INFO: Info    : Started (Tue Apr 23 09:25:54 2019, Wall = 0.000105858s, CPU = 0.002225s, Mem = 5.22266Mb)


[ simnibs ]INFO: Info    :  57 KSP Residual norm 1.206005856898e-04
[ simnibs ]INFO: Info    :  58 KSP Residual norm 1.021544019195e-04
[ simnibs ]INFO: Info    :  59 KSP Residual norm 8.595380854921e-05
[ simnibs ]INFO: Info    :  60 KSP Residual norm 7.182248589085e-05
[ simnibs ]INFO: Info    :  61 KSP Residual norm 5.999882878022e-05
[ simnibs ]INFO: Info    :  62 KSP Residual norm 4.870657872369e-05
[ simnibs ]INFO: Info    :  63 KSP Residual norm 3.957866166326e-05
[ simnibs ]INFO: Info    :  64 KSP Residual norm 3.181850787352e-05
[ simnibs ]INFO: Info    :  65 KSP Residual norm 2.577713828501e-05
[ simnibs ]INFO: Info    :  66 KSP Residual norm 2.169379650757e-05
[ simnibs ]INFO: Info    :  67 KSP Residual norm 1.841695607526e-05
[ simnibs ]INFO: Info    :  68 KSP Residual norm 1.584492689393e-05
[ simnibs ]INFO: Info    :  69 KSP Residual norm 1.336177837640e-05
[ simnibs ]INFO: Info    :  70 KSP Residual norm 1.127905572579e-05
[ simnibs ]INFO: Info    :  71 KSP Residual norm

0


...

[ simnibs ]INFO: Running TMS Position 0
[ simnibs ]INFO: Coil Position Matrix: [[  0.24843005  -0.34948208   0.90340732 -75.53015797]
 [  0.84444724  -0.37876592  -0.37874165  37.14668096]
 [ -0.47454333  -0.85697062  -0.20102234  23.85858958]
 [  0.           0.           0.           1.        ]]
dIdt: 1000000.0

[ simnibs ]INFO: Executing: 
/mnt/tigrlab/projects/jjeyachandra/simnibs/bin/getdp /tmp/tmpFwCKs_/sub-CMH090_TMS_1-0001_Magstim_70mm_Fig8_nii.pro -solve QS -msh /tmp/tmpFwCKs_/tmpmYwnrh.msh -ksp_type cg -ksp_rtol 1e-10 -pc_type icc -pc_factor_levels 2 -v2 -bin
[ simnibs ]INFO: Info    : Running '/mnt/tigrlab/projects/jjeyachandra/simnibs/bin/getdp /tmp/tmpFwCKs_/sub-CMH090_TMS_1-0001_Magstim_70mm_Fig8_nii.pro -solve QS -msh /tmp/tmpFwCKs_/tmpmYwnrh.msh -ksp_type cg -ksp_rtol 1e-10 -pc_type icc -pc_factor_levels 2 -v2 -bin' [GetDP 2.11.3, 1 node, max. 1 thread]
[ simnibs ]INFO: Info    : Started (Tue Apr 23 09:39:39 2019, Wall = 0.000109911s, CPU = 0.002179s, Mem = 5.24609Mb)


[ simnibs ]INFO: Info    :  57 KSP Residual norm 1.265402158935e-04
[ simnibs ]INFO: Info    :  58 KSP Residual norm 9.924737502062e-05
[ simnibs ]INFO: Info    :  59 KSP Residual norm 7.872780984442e-05
[ simnibs ]INFO: Info    :  60 KSP Residual norm 6.491001793329e-05
[ simnibs ]INFO: Info    :  61 KSP Residual norm 5.566785120401e-05
[ simnibs ]INFO: Info    :  62 KSP Residual norm 4.778519463687e-05
[ simnibs ]INFO: Info    :  63 KSP Residual norm 4.169232220987e-05
[ simnibs ]INFO: Info    :  64 KSP Residual norm 3.552937744435e-05
[ simnibs ]INFO: Info    :  65 KSP Residual norm 3.095298197561e-05
[ simnibs ]INFO: Info    :  66 KSP Residual norm 2.690592918940e-05
[ simnibs ]INFO: Info    :  67 KSP Residual norm 2.388393494561e-05
[ simnibs ]INFO: Info    :  68 KSP Residual norm 2.256718400422e-05
[ simnibs ]INFO: Info    :  69 KSP Residual norm 2.172640329583e-05
[ simnibs ]INFO: Info    :  70 KSP Residual norm 2.020202274671e-05
[ simnibs ]INFO: Info    :  71 KSP Residual norm

0


[ simnibs ]INFO: Running TMS Position 0
[ simnibs ]INFO: Coil Position Matrix: [[ -0.56116359   0.5858912    0.58465966 -52.63449192]
 [ -0.81982143  -0.29617808  -0.49007282  52.00757678]
 [  0.11396598   0.75432754  -0.64653052  48.53208174]
 [  0.           0.           0.           1.        ]]
dIdt: 1000000.0

[ simnibs ]INFO: Executing: 
/mnt/tigrlab/projects/jjeyachandra/simnibs/bin/getdp /tmp/tmpBdxiqZ/sub-CMH090_TMS_1-0001_Magstim_70mm_Fig8_nii.pro -solve QS -msh /tmp/tmpBdxiqZ/tmpmQDKMu.msh -ksp_type cg -ksp_rtol 1e-10 -pc_type icc -pc_factor_levels 2 -v2 -bin
[ simnibs ]INFO: Info    : Running '/mnt/tigrlab/projects/jjeyachandra/simnibs/bin/getdp /tmp/tmpBdxiqZ/sub-CMH090_TMS_1-0001_Magstim_70mm_Fig8_nii.pro -solve QS -msh /tmp/tmpBdxiqZ/tmpmQDKMu.msh -ksp_type cg -ksp_rtol 1e-10 -pc_type icc -pc_factor_levels 2 -v2 -bin' [GetDP 2.11.3, 1 node, max. 1 thread]
[ simnibs ]INFO: Info    : Started (Tue Apr 23 09:44:14 2019, Wall = 9.799e-05s, CPU = 0.002302s, Mem = 5.19141Mb)
[ 

[ simnibs ]INFO: Info    :  57 KSP Residual norm 1.352937891028e-04
[ simnibs ]INFO: Info    :  58 KSP Residual norm 1.177087695196e-04
[ simnibs ]INFO: Info    :  59 KSP Residual norm 1.013854327365e-04
[ simnibs ]INFO: Info    :  60 KSP Residual norm 8.817826478038e-05
[ simnibs ]INFO: Info    :  61 KSP Residual norm 8.050378824938e-05
[ simnibs ]INFO: Info    :  62 KSP Residual norm 6.855877212763e-05
[ simnibs ]INFO: Info    :  63 KSP Residual norm 5.876349873331e-05
[ simnibs ]INFO: Info    :  64 KSP Residual norm 4.929412531598e-05
[ simnibs ]INFO: Info    :  65 KSP Residual norm 4.022850298372e-05
[ simnibs ]INFO: Info    :  66 KSP Residual norm 3.491692080840e-05
[ simnibs ]INFO: Info    :  67 KSP Residual norm 3.271432451471e-05
[ simnibs ]INFO: Info    :  68 KSP Residual norm 3.086418141861e-05
[ simnibs ]INFO: Info    :  69 KSP Residual norm 2.904790608457e-05
[ simnibs ]INFO: Info    :  70 KSP Residual norm 2.622112097966e-05
[ simnibs ]INFO: Info    :  71 KSP Residual norm

In [8]:
hist_data = HistoricalData(dim = 3, num_derivatives= 0)
hist_data.append_sample_points([SamplePoint(inp,o,0.0) 
                                for o,inp in 
                                zip(observations,init_pts)])


In [ ]:
#Set up model specifications
prior = DefaultPrior(n_dims = 3 + 2, num_noise=1)
gp_ll = GaussianProcessLogLikelihoodMCMC(historical_data=hist_data,
                                         derivatives=[], prior=prior,
                                         chain_length=1000, burnin_steps=2000,
                                         n_hypers=2**4, noisy=False)
gp_ll.train()

In [ ]:
#Set up optimizer
sgd_params = cGDParams(num_multistarts=200, max_num_steps=50,
                      max_num_restarts=2, num_steps_averaged=4,
                      gamma=0.7, pre_mult=1.0, max_relative_change=0.5,
                      tolerance=1.0e-10)

In [ ]:
#Set up optimization parametes
from IPython.display import clear_output, display
num_iters = 50

In [ ]:
def gen_sample_from_qei(gp,search_domain,sgd_params,num_samples, num_mc=1e4, lhc_iter=2e4):
    
    qEI = ExpectedImprovement(gaussian_process=gp, num_mc_iterations=int(num_mc))
    optimizer = cGDOpt(search_domain, qEI, sgd_params, int(lhc_iter))
    points_to_sample = meio(optimizer, None, num_samples, use_gpu=False, which_gpu=0,
                      max_num_threads=8)
    qEI.set_current_point(points_to_sample[0])
    
    return points_to_sample, qEI.compute_expected_improvement()

In [ ]:
best_point_history = []
for i in np.arange(0,num_iters):
    
    #Optimize qEI and pick samples
    points_to_sample, ei = gen_sample_from_qei(gp_ll.models[0],
                                               c_search_domain, sgd_params=sgd_params,
                                               num_samples=4, num_mc=2**10)
    
    #Evaluate objective function at points
    sampled_points = [-f.evaluate(x,y,t) for x,y,t in points_to_sample]
    evidence = [SamplePoint(c,v,0.0) for c,v in zip(points_to_sample, sampled_points)]
    
    #Update model
    gp_ll.add_sampled_points(evidence)
    gp_ll.train()
    
    #Pull model and pull values
    gp = gp_ll.models[0]
    min_point = np.argmin(gp._points_sampled_value)
    min_val = np.min(gp._points_sampled_value)
    best_coord = gp.get_historical_data_copy().points_sampled[min_point]
    
    clear_output()
    print('Recommended Points:')
    print(points_to_sample)
    print('Expected Improvement: {}'.format(ei))
    print('Current Best:')
    print('f(x*)=',min_val)
    print('Coord:', best_coord)
    
    best_point_history.append(min_val)